**<font color = black size=6>实验十:EM算法</font>**

In [134]:
from collections import Counter
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import random

**<font color = blue size=4>第一部分:实验任务</font>**

<span style="color:purple">=======  
现在给定了一个班级中所有同学的身高数据文件(height.csv)，但不知道各个学生的性别。假设男生身高服从一高斯分布$N_1(\mu_1,\sigma_1^2)$，女生身高服从另一高斯分布$N_2(\mu_2,\sigma_2^2)$，我们可以写出一个混合高斯模型:$x\sim\alpha_1 N_1(\mu_1,\sigma_1^2)+\alpha_2 N_2(\mu_2,\sigma_2^2)$。请使用EM算法完成对该混合高斯分布的估计(即求出对参数$\alpha_1,\alpha_2,\mu_1,\sigma_1,\mu_2,\sigma_2$的估计值)。我们简化地记$\theta_1=(\alpha_1,\mu_1,\sigma_1)$, $\theta_2=(\alpha_2,\mu_2,\sigma_2)$</span>

<span style="color:purple">该数据集(height.csv)特征信息只包括了1个特征，即学生的身高。我们沿用理论课PPT的设置，将隐变量$z_i$按照男生$z_i=1$、女生$z_i=2$的形式进行描述</span>

**<font color = black size=4>E步（Expectation Step）</font>**

<span style="color:purple">1) 将数据集'height.csv'载入并转换为你需要的格式</span>

In [135]:
D = np.array(pd.read_csv('data.csv'))
print(D[:5])


[[170.26930751]
 [179.8860453 ]
 [173.68132906]
 [179.812473  ]
 [174.81481669]]


<span style="color:purple">2)初始化  
初始化$t=0$时的参数($\alpha_1(0)$, $\alpha_2(0)$, $\mu_1(0)$, $\mu_2(0)$, $\sigma_1(0)$, $\sigma_2(0)$). </span>

In [136]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
parameter = np.array([0.5,0.5,150,170,5,5])

<span style="color:purple">3)编写函数P(x,parameter,z)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D,计算每个样本$x_i$的$P(x_i,z_i|\theta)$.</span>

<span style="color:purple">.  
当$z_i=1$时,$$P(x_i,z_i|\theta)=\alpha_1(t)f_1(x_i|\theta_1(t))$$
    当$z_i=2$时,$$P(x_i,z_i|\theta)=\alpha_2(t)f_2(x_i|\theta_2(t))$$</span>

<span style="color:purple"> .   
其中$f_1(x_i|\theta_1(t))$为样本$x_i$在模型$N_1$中的概率密度,公式如下:
    $$f(x_i|\theta_1(t))=\frac{1}{{\sqrt{2\pi}\sigma_1}} e^{-\frac{(x_i-\mu_1)^2}{2\sigma_1^2}}$$</span>

In [137]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def P(x,pa,z):
    if z==1:
        f = pa[0] / (2*math.pi)**0.5*pa[4] * \
            np.exp(-(x-pa[2])**2/(2*pa[4]**2))
    elif z==2:
        f = pa[1] / (2*math.pi)**0.5*pa[5] * \
            np.exp(-(x-pa[3])**2/(2*pa[5]**2))
    return f


<span style="color:purple">4)编写函数Y(x,parameter,z)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D,计算每个样本$x_i$的$y_{1,i}=P((z_i=1)|x_i,\theta)$和$y_{2,i}=P((z_i=2)|x_i,\theta)$.  
公式如下:  
</span>

$$P((z_i=1)|x_i,\theta) = \frac{\alpha_1(t)f_1(x_i|\theta_1(t))}{\alpha_1(t)f_1(x_i|\theta_1(t))+\alpha_2(t)f_2(x_i|\theta_2(t))}$$  
$$P((z_i=2)|x_i,\theta) = \frac{\alpha_2(t)f_2(x_i|\theta_2(t))}{\alpha_1(t)f_1(x_i|\theta_1(t))+\alpha_2(t)f_2(x_i|\theta_2(t))}$$

In [138]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def Y(x,pa,z):
    f1 = P(x,pa,1); f2 = P(x,pa,2)
    deno = pa[0]*f1 + pa[1]*f2
    if z==1:
        y = pa[0]*f1 / deno
    elif z==2:
        y = pa[1]*f2 / deno
    return y

<span style="color:purple">5)编写函数Q(x,parameter)  
 计算对数似然函数在该分布和基于$\theta(t)$下的期望值$Q(\theta)$.单个样本的期望值公式如下:$$E_{z_i}logP(x_i,z_i|\theta)=y_{1,i}log(\alpha_1(t)f_1(x_i|\theta_1(t)))+y_{2,i}log(\alpha_2(t)f_2(x_i|\theta_2(t)))$$</span>

In [139]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def Q(x,pa):
    y1 = Y(x,pa,1); y2 = Y(x,pa,2)
    f1 = P(x,pa,1); f2 = P(x,pa,2)
    E = y1*np.log(pa[0]*f1) + y2*np.log(pa[1]*f2)
    return E

**<font color = black size=4>M步 (Maximization Step)</font>**

<span style="color:purple">6)编写函数alpha_expection(D,parameter)  
 给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\alpha_1(t+1),\alpha_2(t+1))$的更新值.
</span>

$$\alpha_1(t+1)=\frac{\sum_{i=1}^m{y_{1,i}}}{m}$$  
$$\alpha_2(t+1)=\frac{\sum_{i=1}^m{y_{2,i}}}{m}$$

In [140]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def alpha_expection(D,pa):
    alpha_next_1 = np.sum(Y(D,pa,1)) / len(D)
    alpha_next_2 = np.sum(Y(D,pa,2)) / len(D)
    return alpha_next_1, alpha_next_2

<span style="color:purple">7)编写函数mu_expection(D,parameter)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\mu_1(t+1),\mu_2(t+1))$的更新值.
</span>

$$\mu_1(t+1)=\frac{\sum_{i=1}^m{y_{1,i}x_i}}{\sum_{i=1}^m{y_{1,i}}}$$
$$\mu_2(t+1)=\frac{\sum_{i=1}^m{y_{2,i}x_i}}{\sum_{i=1}^m{y_{2,i}}}$$

In [141]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def mu_expection(D,pa):
    Y1 = Y(D,pa,1); Y2 = Y(D,pa,2)
    mu_next_1 = np.sum(Y1*D) / np.sum(Y1)
    mu_next_2 = np.sum(Y2*D) / np.sum(Y2)
    return mu_next_1, mu_next_2


<span style="color:purple">8)编写函数sigma_expection(D,parameter,mu_next_1,mu_next_2)  
给定参数$(\alpha_1(t),\alpha_2(t),\mu_1(t),\mu_2(t),\sigma_1(t),\sigma_2(t))$以及数据集D，计算第$(t+1)$轮的$(\sigma_1(t+1),\sigma_2(t+1))$的更新值.
</span>

$$\sigma_1(t+1)=\sqrt{\frac{\sum_{i=1}^m{y_{1,i}(x_i-\mu_1(t+1))^2}}{\sum_{i=1}^m{y_{1,i}}}}$$
$$\sigma_2(t+1)=\sqrt{\frac{\sum_{i=1}^m{y_{2,i}(x_i-\mu_2(t+1))^2}}{\sum_{i=1}^m{y_{2,i}}}}$$

In [142]:
#parameter=[alpha1,alpha2,mu1,mu2,sigma1,sigma2]
def sigma_expection(D,pa,mu_next_1,mu_next_2):
    Y1 = Y(D,pa,1); Y2 = Y(D,pa,2)
    sigma_next_1 = (np.sum(Y1*(D-mu_next_1)**2)/np.sum(Y1))**0.5
    sigma_next_2 = (np.sum(Y2*(D-mu_next_2)**2)/np.sum(Y2))**0.5
    return sigma_next_1, sigma_next_2

**<font color = black size=4>E步和M步的迭代过程</font>**

<span style="color:purple">9) 利用前面编写的函数完成EM算法的迭代过程，直至达到收敛要求。请至少完成【3次】不同的初始值下的迭代过程，并比较选出最好的。  
    收敛要求给出如下几种参考:  
    1.迭代轮数达到指定轮数;  
    2.每轮参数更新的差值小于阈值.</span>

<img src='./EM Algorithm Pseudocode.png'>

.  
我们给出这个数据集的正确相关信息作为参考:$\theta_1:(\alpha_1=0.625,\mu_1=175,\sigma_1=4)$,$\theta_2:(\alpha_2=0.375,\mu_2=165,\sigma_2=6)$

In [143]:
final_pa = np.zeros((6))
threshold = 1e-6

parameters = [
    [0.5,0.5,150,170,5,5],
    [0.3,0.9,150,170,1,2],
    [0.8,0.2,130,190,20,30]
]

def test(parameter,threshold):
    for i in range(100):
        final_pa = parameter.copy()

        #exception = Q(D,parameter)

        parameter[0], parameter[1] = alpha_expection(D,final_pa)
        if(parameter[1] < 0.01):
            parameter[1] = 1.0
        if(parameter[0] < 0.01):
            parameter[0] = 1
        parameter[2], parameter[3] = mu_expection(D, final_pa)
        parameter[4], parameter[5] = sigma_expection(D, final_pa, parameter[2], parameter[3])
        print(f"pre_pa:", [f"{num:.3f}" for num in final_pa])
        print(f"new_pa:", [f"{num:.3f}" for num in parameter])
        
        if abs(parameter[0] - final_pa[0]) < 1e-6 and abs(parameter[1] - final_pa[1]) < 1e-6 and abs(parameter[2] - final_pa[2]) < 1e-6 \
        and abs(parameter[3] - final_pa[3]) < 1e-6 and abs(parameter[4] - final_pa[4]) < 1e-6 and abs(parameter[5] - final_pa[5]) < 1e-6:
            break
     
# test(parameters[0],threshold)   
# test(parameters[1],threshold)   
test(parameters[2],threshold)   


pre_pa: ['0.800', '0.200', '130.000', '190.000', '20.000', '30.000']
new_pa: ['0.576', '0.424', '169.461', '174.367', '6.926', '5.645']
pre_pa: ['0.576', '0.424', '169.461', '174.367', '6.926', '5.645']
new_pa: ['0.712', '0.288', '170.188', '174.897', '7.072', '4.879']
pre_pa: ['0.712', '0.288', '170.188', '174.897', '7.072', '4.879']
new_pa: ['0.915', '0.085', '171.178', '175.484', '6.943', '4.146']
pre_pa: ['0.915', '0.085', '171.178', '175.484', '6.943', '4.146']
new_pa: ['0.996', '1.000', '171.528', '175.811', '6.861', '3.554']
pre_pa: ['0.996', '1.000', '171.528', '175.811', '6.861', '3.554']
new_pa: ['0.787', '0.213', '170.394', '175.802', '7.106', '3.327']
pre_pa: ['0.787', '0.213', '170.394', '175.802', '7.106', '3.327']
new_pa: ['0.979', '0.021', '171.444', '176.131', '6.883', '2.938']
pre_pa: ['0.979', '0.021', '171.444', '176.131', '6.883', '2.938']
new_pa: ['1.000', '1.000', '171.543', '176.270', '6.857', '2.644']
pre_pa: ['1.000', '1.000', '171.543', '176.270', '6.857', '2

**<font color = blue size=4>第二部分:作业提交</font>**

一、实验课下课前提交完成代码，如果下课前未完成，请将已经完成的部分进行提交，未完成的部分于之后的实验报告中进行补充  
要求:  
1)文件格式为：学号-姓名.ipynb  
2)【不要】提交文件夹、压缩包、数据集等无关文件，只需提交单个ipynb文件即可，如果交错请到讲台前联系助教，删掉之前的错误版本后再进行提交

二、实验报告截止日期： 【11月24日 14:20】
要求：  
1)文件格式为：学号-姓名.pdf  
2)【不要】提交文件夹、压缩包、代码文件、数据集等任何与实验报告无关的文件，只需要提交单个pdf文件即可  
3)文件命名时不需要额外添加“实验几”等额外信息，按照格式提交  
4)每周的实验报告提交地址会变化，且有时间限制，提交时间为下周的实验课开始时，请注意及时提交。

实验十(EM算法)的实验报告上交地址:https://send2me.cn/9UjusmMn/S_Cz3o_FpKQEsA  

三、课堂课件获取地址:https://www.jianguoyun.com/p/DZKTh-IQp5WhChiIn6gFIAA  
实验内容获取地址:https://www.jianguoyun.com/p/DbCHB9wQp5WhChiKn6gFIAA